# Graph Enhanced RAG

In [3]:
import os
import json
import re
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from neo4j import GraphDatabase
import time

In [4]:
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Embedding model 설정
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small', api_key=OPENAI_API_KEY)
embedding_dimension = 1536 # text-embedding-3-small 차원

# Data path
pdf_path = './dataset/criminal-law.pdf'
precedent_dir = './dataset/precedent_label/'

In [6]:
# Load PDF
loader = PyPDFLoader(pdf_path)
# pages = loader.load()[2:] # 첫 두 페이지 생략 (표지랑 목차)
pages = loader.load() # 아님말고
full_text = "\n".join(page.page_content for page in pages)

# 전체 텍스트에서 모든 조항 시작 위치 찾기
article_pattern = r'제\d+조(?:의\d+)?(?:\s*\(.+?\))?'
matches = list(re.finditer(article_pattern, full_text))

articles = {}
for i in range(len(matches)):
  current_match = matches[i]
  current_article_id = current_match.group(0).strip() # 현재 조항 ID
  
  # 현재 조항 시작 위치
  start_pos = current_match.start()
  
  # 다음 조항 시작 위치 (없으면 텍스트 끝까지)
  end_pos = matches[i+1].start() if i < len(matches)-1 else len(full_text)
  
  # 현재 조항의 전체 내용 (ID 포함)
  article_text = full_text[start_pos:end_pos].strip()
  
  # 저장 (ID는 조항 번호만)
  articles[current_article_id] = article_text
  
print(f"Processed {len(articles)} article from PDF")

# 예시 출력
if articles:
  article_ids = list(articles.keys())
  
  print("\n--- 처음 5개 조항 ---")
  for i in range(min(5, len(article_ids))):
    article_id = article_ids[i]
    content = articles[article_id]
    print(f"\n--- Article: {article_id} ---")
    print(content[:200] + "..." if len(content) > 200 else content)
    
  print("\n--- 마지막 5개 조항 ---")
  for i in range(max(0, len(article_ids)-10), len(article_ids)):
    article_id = article_ids[i]
    content = articles[article_id]
    print(f"\n--- Article: {article_id} ---")
    print(content[:200] + "..." if len(content) > 200 else content)

Processed 548 article from PDF

--- 처음 5개 조항 ---

--- Article: 제1조(범죄의 성립과 처벌) ---
제1조(범죄의 성립과 처벌) ①범죄의 성립과 처벌은 행위 시의 법률에 의한다.
②범죄 후 법률의 변경에 의하여 그 행위가 범죄를 구성하지 아니하거나 형이 구법보다 경한
때에는 신법에 의한다.
③재판확정 후 법률의 변경에 의하여 그 행위가 범죄를 구성하지 아니하는 때에는 형의 집행
을 면제한다.

--- Article: 제2조(국내범) ---
제2조(국내범) 본법은 대한민국영역 내에서 죄를 범한 내국인과 외국인에게 적용한다.

--- Article: 제3조(내국인의 국외범) ---
제3조(내국인의 국외범) 본법은 대한민국영역 외에서 죄를 범한 내국인에게 적용한다.

--- Article: 제4조(국외에 있는 내국선박 등에서 외국인이 범한 죄) ---
제4조(국외에 있는 내국선박 등에서 외국인이 범한 죄) 본법은 대한민국영역 외에 있는 대한민
국의 선박 또는 항공기 내에서 죄를 범한 외국인에게 적용한다.

--- Article: 제5조(외국인의 국외범) ---
제5조(외국인의 국외범) 본법은 대한민국영역 외에서 다음에 기재한 죄를 범한 외국인에게 적용
한다.
1. 내란의 죄
2. 외환의 죄
3. 국기에 관한 죄
4. 통화에 관한 죄
5. 유가증권, 우표와 인지에 관한 죄
6. 문서에 관한 죄 중

--- 마지막 5개 조항 ---

--- Article: 제4조 (형에 관한 경과조치) ---
제4조 (형에 관한 경과조치) 이 법 시행전에 종전의 형법규정에 의하여 형의 선고를 받은 자는
이 법에 의하여 형의 선고를 받은 것으로 본다. 집행유예 또는 선고유예를 받은 경우에도 이와
같다.

--- Article: 제5조 (다른 법령과의 관계) ---
제5조 (다른 법령과의 관계) 이 법 시행당시 다른 법령에서 종전의 형법 규정(장의 제목을 포함
한다)을 인용하고 있는 경우에 이 법중 그에 해당하는 규정이 있는 때에는 종전의 

In [ ]:
# Load precedent JSON files (판례 불러오기)
precedents = []
for filename in os.listdir(precedent_dir):
    if filename.endswith(".json"):
        filepath = os.path.join(precedent_dir, filename)
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # 기존에 라벨링 되어있었음
                precedent_info = {
                    "case_id": data.get("info", {}).get("caseNoID", filename.replace(".json", "")), # 사건번호 (없으면 파일명 사용)
                    "case_name": data.get("info", {}).get("caseNm"), # 사건명
                    "judgment_summary": data.get("jdgmn"), # 판결 요약
                    "full_summary": " ".join([s.get("summ_contxt", "") for s in data.get("Summary", [])]), # 전체 요약 텍스트
                    "keywords": [kw.get("keyword") for kw in data.get("keyword_tagg", []) if kw.get("keyword")], # 키워드 목록
                    "referenced_rules": data.get("Reference_info", {}).get("reference_rules", "").split(',') if data.get("Reference_info", {}).get("reference_rules") else [], # 참조 법조항
                    "referenced_cases": data.get("Reference_info", {}).get("reference_court_case", "").split(',') if data.get("Reference_info", {}).get("reference_court_case") else [], # 참조 판례
                }
                # 참조 법조항 정제 (조항 번호만)
                cleaned_rules = []
                rule_pattern = re.compile(r'제\d+조(?:의\d+)?') # 패턴 찾기: "제X조" or "제X조의Y"
                for rule in precedent_info["referenced_rules"]:
                    # 각 규칙 문자열에서 모든 일치 항목 찾기
                    matches = rule_pattern.findall(rule.strip())
                    cleaned_rules.extend(matches)
                precedent_info["referenced_rules"] = list(set(cleaned_rules)) # 중복 제거하여 고유한 조항 번호만 유지

                precedents.append(precedent_info)
        except json.JSONDecodeError:
            print(f"Warning: Could not decode JSON from {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")


print(f"Loaded {len(precedents)} precedents.")
# 예시 출력
if precedents:
    print("\n--- Example Precedent ---")
    print(json.dumps(precedents[0], indent=2, ensure_ascii=False))

In [ ]:
# 로드된 판례 중 무작위로 1,000개만 선택 (시간 문제 때문에...)
import random
random.seed(42)  # 재현성을 위한 시드 설정

# 전체 판례 수 저장
total_precedents = len(precedents)

# 무작위로 1,000개 선택 (또는 전체 판례 수가 1,000개보다 적다면 모두 선택)
sample_size = min(1000, total_precedents)
precedents = random.sample(precedents, sample_size)

In [10]:
# Neo4j 데이터베이스에 연결
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    driver.verify_connectivity()  # 연결 확인
    print("Successfully connected to Neo4j.")
except Exception as e:
    print(f"Failed to connect to Neo4j: {e}")
    # 연결 실패 시 실행 중단
    raise

# 빠른 조회와 임베딩 검색을 위한 제약조건과 인덱스 생성 함수
def setup_neo4j(driver, dimension):
    with driver.session(database="neo4j") as session:
        # 고유성을 위한 제약조건 설정
        session.run("CREATE CONSTRAINT article_id IF NOT EXISTS FOR (a:Article) REQUIRE a.id IS UNIQUE")  # 법조항 ID 고유성 제약
        session.run("CREATE CONSTRAINT precedent_id IF NOT EXISTS FOR (p:Precedent) REQUIRE p.id IS UNIQUE")  # 판례 ID 고유성 제약
        session.run("CREATE CONSTRAINT keyword_text IF NOT EXISTS FOR (k:Keyword) REQUIRE k.text IS UNIQUE")  # 키워드 텍스트 고유성 제약

        # 법조항(Article)에 대한 벡터 인덱스 생성
        try:
            session.run(
                "CREATE VECTOR INDEX article_embedding IF NOT EXISTS "  # 존재하지 않는 경우에만 생성
                "FOR (a:Article) ON (a.embedding) "  # Article 노드의 embedding 속성에 대한 인덱스
                f"OPTIONS {{indexConfig: {{`vector.dimensions`: {dimension}, `vector.similarity_function`: 'cosine'}}}}"  # 벡터 차원 및 유사도 함수 설정
            )
            print("Article vector index created or already exists.")
        except Exception as e:
            print(f"Error creating Article vector index (may require Neo4j 5.11+ and APOC): {e}")  # Neo4j 버전 문제일 수 있음
            print("Continuing without vector index creation for Article.")  # 인덱스 없이 계속 진행

        # 판례(Precedent)에 대한 벡터 인덱스 생성
        try:
            session.run(
                "CREATE VECTOR INDEX precedent_embedding IF NOT EXISTS "  # 존재하지 않는 경우에만 생성
                "FOR (p:Precedent) ON (p.embedding) "  # Precedent 노드의 embedding 속성에 대한 인덱스
                f"OPTIONS {{indexConfig: {{`vector.dimensions`: {dimension}, `vector.similarity_function`: 'cosine'}}}}"  # 벡터 차원 및 유사도 함수 설정
            )
            print("Precedent vector index created or already exists.")
        except Exception as e:
            print(f"Error creating Precedent vector index (may require Neo4j 5.11+ and APOC): {e}")  # Neo4j 버전 문제일 수 있음
            print("Continuing without vector index creation for Precedent.")  # 인덱스 없이 계속 진행

        # 인덱스가 활성화될 때까지 대기 (중요!)
        print("Waiting for indexes to populate...")
        session.run("CALL db.awaitIndexes(300)")  # 최대 300초(5분) 동안 대기
        print("Indexes should be online.")  # 인덱스 활성화 완료


setup_neo4j(driver, embedding_dimension)  # 설정 함수 호출, embedding_dimension은 임베딩 벡터의 차원 크기

Successfully connected to Neo4j.
Article vector index created or already exists.
Precedent vector index created or already exists.
Waiting for indexes to populate...
Indexes should be online.


In [ ]:
# 법조항 노드 생성 및 임베딩 생성/저장
def create_article_nodes(driver, articles_dict, embed_model):
    print(f"Creating {len(articles_dict)} Article nodes...")  # 생성할 법조항 노드 수 출력
    count = 0
    start_time = time.time()  # 실행 시간 측정 시작
    with driver.session(database="neo4j") as session:
        for article_id, content in articles_dict.items():
            if not content:  # 내용이 비어있는 경우 건너뛰기
                print(f"Skipping article {article_id} due to empty content.")
                continue
            try:
                # 텍스트에 대한 임베딩 생성
                embedding = embed_model.embed_query(content)

                # Neo4j에 노드 생성
                session.run(
                    """
                    MERGE (a:Article {id: $article_id})  # 해당 ID의 법조항이 있으면 찾고, 없으면 생성
                    SET a.text = $content,               # 법조항 텍스트 설정
                        a.embedding = $embedding         # 임베딩 벡터 설정
                    """,
                    article_id=article_id,
                    content=content,
                    embedding=embedding
                )
                count += 1
                if count % 50 == 0:  # 50개마다 진행상황 출력
                    print(f"  Processed {count}/{len(articles_dict)} articles...")
            except Exception as e:
                print(f"Error processing article {article_id}: {e}")  # 오류 발생 시 메시지 출력

    end_time = time.time()  # 실행 시간 측정 종료
    print(f"Finished creating {count} Article nodes in {end_time - start_time:.2f} seconds.")  # 총 처리 시간 출력

create_article_nodes(driver, articles, embedding_model)  # 함수 실행: 법조항 노드 생성 및 임베딩 저장

In [ ]:
# 판례 노드, 키워드 노드 생성 및 관계 설정
def create_precedent_nodes_and_relationships(driver, precedents_list, embed_model):
    print(f"Creating {len(precedents_list)} Precedent nodes and relationships...")  # 생성할 판례 노드 수 출력
    count = 0
    start_time = time.time()  # 실행 시간 측정 시작
    with driver.session(database="neo4j") as session:
        for precedent in precedents_list:
            # 임베딩에 사용할 텍스트: 전체 요약이 있으면 사용, 없으면 판결 요약 사용
            text_to_embed = precedent.get("full_summary") or precedent.get("judgment_summary")
            if not text_to_embed:
                print(f"Skipping precedent {precedent.get('case_id')} due to empty summary.")  # 요약이 없는 경우 건너뛰기
                continue

            try:
                # 텍스트 임베딩 생성
                embedding = embed_model.embed_query(text_to_embed)

                # 판례 노드 생성
                session.run(
                    """
                    MERGE (p:Precedent {id: $case_id})  # 해당 ID의 판례가 있으면 찾고, 없으면 생성
                    SET p.name = $case_name,            # 판례명 설정
                        p.judgment_summary = $judgment_summary,  # 판결 요약 설정
                        p.full_summary = $full_summary,          # 전체 요약 설정
                        p.embedding = $embedding         # 임베딩 벡터 설정
                    """,
                    case_id=precedent["case_id"],
                    case_name=precedent["case_name"],
                    judgment_summary=precedent["judgment_summary"],
                    full_summary=precedent["full_summary"],
                    embedding=embedding
                )

                # 키워드 노드 생성 및 판례와의 관계 설정
                for keyword_text in precedent["keywords"]:
                    session.run(
                        """
                        MERGE (k:Keyword {text: $keyword_text})  # 키워드 노드 생성 또는 찾기
                        WITH k
                        MATCH (p:Precedent {id: $case_id})       # 판례 노드 찾기
                        MERGE (p)-[:HAS_KEYWORD]->(k)            # 판례와 키워드 간 관계 생성
                        """,
                        keyword_text=keyword_text,
                        case_id=precedent["case_id"]
                    )

                # 참조된 법조항과의 관계 생성
                # 참고: 앞서 추출한 정제된 법조항 ID를 사용합니다
                # "제X조" 형식을 기반으로 매칭합니다.
                for article_id_ref in precedent["referenced_rules"]:
                     # 참조된 ID로 시작하는 법조항 노드 찾기(예: "제21조"는 "제21조(정당방위)"와 매칭됨)
                     # 정확한 제목이 참조에 없는 경우에도 매칭이 가능하도록 덜 정밀한 방식 사용
                    session.run(
                        """
                        MATCH (p:Precedent {id: $case_id})         # 판례 노드 찾기
                        MATCH (a:Article)                          # 모든 법조항 노드 찾기
                        WHERE a.id STARTS WITH $article_id_ref     # 특정 ID로 시작하는 법조항만 필터링
                        MERGE (p)-[:REFERENCES_ARTICLE]->(a)       # 판례가 법조항을 참조하는 관계 생성
                        """,
                        case_id=precedent["case_id"],
                        article_id_ref=article_id_ref  # 추출된 "제X조" 사용
                    )

                # 선택사항: 다른 참조된 판례와의 관계 생성 (필요한 경우)
                # for ref_case_id in precedent["referenced_cases"]:
                #    session.run(...) # MERGE (p)-[:REFERENCES_CASE]->(other_p:Precedent {id: ref_case_id})

                count += 1
                if count % 100 == 0:  # 100개마다 진행상황 출력
                    print(f"  Processed {count}/{len(precedents_list)} precedents...")

            except Exception as e:
                print(f"Error processing precedent {precedent.get('case_id')}: {e}")  # 오류 발생 시 메시지 출력

    end_time = time.time()  # 실행 시간 측정 종료
    print(f"Finished creating {count} Precedent nodes and relationships in {end_time - start_time:.2f} seconds.")  # 총 처리 시간 출력


create_precedent_nodes_and_relationships(driver, precedents, embedding_model)  # 함수 실행: 판례 노드 생성 및 관계 설정

# 작업 완료 후 드라이버 연결 종료
# driver.close()  # 다음 단계에서 쿼리를 위해 연결 상태 유지

In [8]:
def graph_enhanced_rag(driver, query_text, embed_model, top_k=3):
    print(f"\n--- 그래프 기반 검색 실행: '{query_text}' ---")
    start_time = time.time()

    # 임베딩 생성
    query_embedding = embed_model.embed_query(query_text)
    
    # 키워드 추출
    keywords = [w for w in re.findall(r'\w+', query_text) if len(w) > 1]
    
    results = []
    with driver.session(database="neo4j") as session:
        try:
            # 그래프 구조를 활용한 검색
            cypher_query = """
            // 1. 벡터 검색으로 시작 법조항 찾기
            CALL db.index.vector.queryNodes('article_embedding', 5, $query_embedding) 
            YIELD node as article, score as article_score
            
            // 2. 해당 법조항과 연결된 판례와 키워드 찾기
            OPTIONAL MATCH (precedent:Precedent)-[:REFERENCES_ARTICLE]->(article)
            OPTIONAL MATCH (precedent)-[:HAS_KEYWORD]->(keyword:Keyword)
            
            // 3. 결과 집계 및 점수 계산
            WITH article, article_score, precedent, 
                 collect(DISTINCT keyword.text) as keywords,
                 count(precedent) as precedent_count
            
            // 법조항 점수 = 벡터 점수 + 판례 인용 수에 따른 보너스
            WITH article, article_score + (precedent_count * 0.01) as final_score,
                 precedent_count, keywords
            
            RETURN article.id as id, 
                   'Article' as type, 
                   article.text as text, 
                   final_score as score,
                   precedent_count,
                   keywords
            ORDER BY final_score DESC
            LIMIT $article_limit
            """
            
            # 법조항 검색
            article_results = session.run(
                cypher_query,
                query_embedding=query_embedding,
                article_limit=top_k
            )
            
            for record in article_results:
                results.append({
                    "type": record["type"],
                    "id": record["id"],
                    "score": record["score"],
                    "text": record["text"][:300] + "..." if len(record["text"]) > 300 else record["text"],
                    "precedent_count": record["precedent_count"],
                    "related_keywords": record["keywords"]
                })
            
            # 관련 판례 검색
            for article_result in results[:3]:  # 상위 3개 법조항에 대해서만
                if article_result["type"] == "Article":
                    precedent_query = """
                    // 1. 특정 법조항을 참조하는 판례 찾기
                    MATCH (precedent:Precedent)-[:REFERENCES_ARTICLE]->(article:Article)
                    WHERE article.id STARTS WITH $article_id
                    
                    // 2. 해당 판례와 키워드
                    OPTIONAL MATCH (precedent)-[:HAS_KEYWORD]->(keyword:Keyword)
                    
                    // 3. 벡터 유사도 계산
                    CALL db.index.vector.queryNodes('precedent_embedding', 20, $query_embedding) 
                    YIELD node as vector_node, score as vector_score
                    WHERE precedent = vector_node
                    
                    // 4. 검색어와 관련된 키워드가 있는지 확인하여 보너스 점수
                    WITH precedent, vector_score, 
                         collect(DISTINCT keyword.text) as keywords,
                         sum(CASE WHEN $query_keywords IS NULL THEN 0
                              WHEN any(k IN $query_keywords WHERE keyword.text CONTAINS k) 
                              THEN 0.05 ELSE 0 END) as keyword_bonus
                    
                    // 5. 다른 법조항도 참조하는지 확인
                    MATCH (precedent)-[:REFERENCES_ARTICLE]->(ref_article:Article)
                    
                    // 6. 최종 결과 반환
                    RETURN precedent.id as id,
                           'Precedent' as type,
                           precedent.name as name,
                           precedent.full_summary as text,
                           vector_score + keyword_bonus as score,
                           keywords,
                           collect(DISTINCT ref_article.id) as referenced_articles
                    ORDER BY score DESC
                    LIMIT 2
                    """
                    
                    precedent_results = session.run(
                        precedent_query,
                        article_id=article_result["id"],
                        query_embedding=query_embedding,
                        query_keywords=keywords
                    )
                    
                    for record in precedent_results:
                        # 중복 제거
                        if not any(r["type"] == "Precedent" and r["id"] == record["id"] for r in results):
                            results.append({
                                "type": record["type"],
                                "id": record["id"],
                                "name": record["name"],
                                "score": record["score"],
                                "text": record["text"][:300] + "..." if len(record["text"]) > 300 else record["text"],
                                "keywords": record["keywords"],
                                "referenced_articles": record["referenced_articles"]
                            })
        
        except Exception as e:
            print(f"그래프 검색 오류: {e}")
            # 백업: 기본 벡터 검색
            try:
                # Article 검색
                article_res = session.run(
                    """
                    CALL db.index.vector.queryNodes('article_embedding', $top_k, $query_embedding) 
                    YIELD node, score
                    RETURN node.id AS id, 'Article' as type, node.text AS text, score
                    """,
                    top_k=top_k,
                    query_embedding=query_embedding
                )
                
                for record in article_res:
                    results.append({
                        "type": record["type"],
                        "id": record["id"],
                        "score": record["score"],
                        "text": record["text"][:300] + "..." if len(record["text"]) > 300 else record["text"]
                    })
                
                # Precedent 검색
                precedent_res = session.run(
                    """
                    CALL db.index.vector.queryNodes('precedent_embedding', $top_k, $query_embedding) 
                    YIELD node, score
                    MATCH (node)-[:REFERENCES_ARTICLE]->(a:Article)
                    OPTIONAL MATCH (node)-[:HAS_KEYWORD]->(k:Keyword)
                    RETURN node.id AS id, 'Precedent' as type, 
                           node.name AS name, node.full_summary AS text, 
                           score,
                           collect(DISTINCT a.id) as referenced_articles,
                           collect(DISTINCT k.text) as keywords
                    """,
                    top_k=top_k,
                    query_embedding=query_embedding
                )
                
                for record in precedent_res:
                    results.append({
                        "type": record["type"],
                        "id": record["id"],
                        "name": record["name"],
                        "score": record["score"],
                        "text": record["text"][:300] + "..." if len(record["text"]) > 300 else record["text"],
                        "referenced_articles": record["referenced_articles"],
                        "keywords": record["keywords"]
                    })
            except Exception as e2:
                print(f"백업 검색 오류: {e2}")
    
    end_time = time.time()
    print(f"검색 완료: {end_time - start_time:.2f}초 소요")

    # 결과를 스코어로 정렬
    results.sort(key=lambda x: x["score"], reverse=True)

    print("\n--- 검색 결과 ---")
    for i, res in enumerate(results[:top_k]):
        print(f"{i+1}. 유형: {res['type']}, ID: {res['id']}, 스코어: {res['score']:.4f}")
        if res['type'] == 'Precedent':
            print(f"   이름: {res.get('name')}")
            print(f"   키워드: {res.get('keywords')}")
            print(f"   참조 법조항: {res.get('referenced_articles')}")
        elif res['type'] == 'Article':
            print(f"   관련 판례 수: {res.get('precedent_count', 0)}")
            print(f"   관련 키워드: {res.get('related_keywords')}")
        print(f"   미리보기: {res['text']}")
        print("-" * 20)

    return results[:top_k]

In [11]:
search_function = graph_enhanced_rag 

# 테스트 쿼리
query = "정당방위의 요건은 무엇인가?"
retrieved_context = search_function(driver, query, embedding_model, top_k=3)

# 드라이버 연결 종료
driver.close()
print("\nNeo4j 드라이버 연결 종료")


--- 그래프 기반 검색 실행: '정당방위의 요건은 무엇인가?' ---


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n            // 1. 벡터 검색으로 시작 법조항 찾기\n            CALL db.index.vector.queryNodes('article_embedding', 5, $query_embedding) \n            YIELD node as article, score as article_score\n            \n            // 2. 해당 법조항과 연결된 판례와 키워드 찾기\n            OPTIONAL MATCH (precedent:Precedent)-[:REFERENCES_ARTICLE]->(article)\n            OPTIONAL MATCH (precedent)-[:HAS_KEYWORD]->(keyword:Keyword)\n            \n            // 3. 결과 집계 및 점수 계산\n            WITH article, article_score, precedent, \n                 collect(DISTINCT keyword.text) as keywords,\n                 count(precedent) as precedent_count\n            \n            // 법조항 점수 = 벡터 점수 + 판례 인용 수에 따른 보너스\n            W

검색 완료: 2.81초 소요

--- 검색 결과 ---
1. 유형: Article, ID: 제21조(정당방위), 스코어: 0.7723
   관련 판례 수: 7
   관련 키워드: ['공소시효', '정지', '연장', '배제', '특례조항', '소급적용', '경과규정']
   미리보기: 제21조(정당방위) ①자기 또는 타인의 법익에 대한 현재의 부당한 침해를 방위하기 위한 행위는
상당한 이유가 있는 때에는 벌하지 아니한다.
②방위행위가 그 정도를 초과한 때에는 정황에 의하여 그 형을 감경 또는 면제할 수 있다.
③전항의 경우에 그 행위가 야간 기타 불안스러운 상태하에서 공포, 경악, 흥분 또는 당황으로
인한 때에는 벌하지 아니한다.
--------------------
2. 유형: Article, ID: 제21조(정당방위), 스코어: 0.7623
   관련 판례 수: 6
   관련 키워드: ['성범죄', '선고형', '경합범', '성폭력처벌법', '개정', '신상정보 등록기간']
   미리보기: 제21조(정당방위) ①자기 또는 타인의 법익에 대한 현재의 부당한 침해를 방위하기 위한 행위는
상당한 이유가 있는 때에는 벌하지 아니한다.
②방위행위가 그 정도를 초과한 때에는 정황에 의하여 그 형을 감경 또는 면제할 수 있다.
③전항의 경우에 그 행위가 야간 기타 불안스러운 상태하에서 공포, 경악, 흥분 또는 당황으로
인한 때에는 벌하지 아니한다.
--------------------
3. 유형: Precedent, ID: 92도2540, 스코어: 0.7386
   이름: 살인
   키워드: ['타인의 법익', '상당한 이유']
   참조 법조항: ['제10조(심신장애자)', '제21조(정당방위)', '제308조(사자의 명예훼손)', '제308조', '제10조 (폐지되는 법률등)']
   미리보기: 정당방위의 성립요건으로서의 방어행위에는 순수한 수비적 방어뿐 아니라 적극적 반격을 포함하는 반격방어의 형태도 포함됨은 소론과 같다고 하겠으나, 그 방어행위는 자기 또는 타인

In [14]:
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from openai import OpenAI

In [15]:
# 검색 결과를 위한 타입 정의
class SearchResult(BaseModel):
    type: str
    id: str
    score: float
    text: str
    name: Optional[str] = None
    keywords: Optional[List[str]] = None
    referenced_articles: Optional[List[str]] = None
    precedent_count: Optional[int] = None
    related_keywords: Optional[List[str]] = None

# 검색 함수를 호출할 때 전달될 컨텍스트
@dataclass
class LegalContext:
    query: str
    driver: Any  # Neo4j driver
    embedding_model: Any

# Agent 응답 결과 모델
class LegalResponse(BaseModel):
    answer: str
    reasoning: str
    sources: List[str]

# Agent 생성
legal_agent = Agent(
    'openai:gpt-4o-mini',  # 요구사항에 맞는 모델
    deps_type=LegalContext,
    result_type=LegalResponse,
    system_prompt="""
    당신은 한국 형법 전문가입니다. 법조항과 판례를 바탕으로 정확한 법률 해석과 설명을 제공해야 합니다.
    
    사용자의 질문을 분석하고, 검색 도구를 사용해 관련 법조항과 판례를 찾아 답변을 작성하세요.
    항상 출처를 명확히 인용하고, 근거를 제시하며, 객관적이고 사실에 기반한 답변을 제공하세요.
    
    답변 형식:
    1. 직접적인 질문 답변: 사용자 질문에 명확하게 답변
    2. 법적 근거: 관련 법조항 및 판례 인용
    3. 추가 설명: 필요시 법적 개념과 적용에 대한 상세 설명 제공
    
    답변할 수 없는 질문이나 법적 조언이 필요한 경우 한계를 분명히 밝히세요.
    """,
)

# 검색 도구 등록
@legal_agent.tool
async def search_legal_knowledge(ctx: RunContext[LegalContext], query: str) -> List[SearchResult]:
    """
    법률 지식 그래프에서 관련 법조항과 판례를 검색합니다.
    
    Args:
        query: 검색할 질문이나 키워드
        
    Returns:
        관련된 법조항과 판례 목록
    """
    # 실제 검색 함수 호출 (동기 함수이므로 변환 필요)
    raw_results = graph_enhanced_rag(
        ctx.deps.driver, 
        query or ctx.deps.query, 
        ctx.deps.embedding_model,
        top_k=5
    )
    
    # 결과를 SearchResult 객체로 변환
    results = []
    for result in raw_results:
        # 일부 필드가 없을 수 있으므로 안전하게 처리
        search_result = SearchResult(
            type=result.get("type", ""),
            id=result.get("id", ""),
            score=result.get("score", 0.0),
            text=result.get("text", ""),
            name=result.get("name"),
            keywords=result.get("keywords", []),
            referenced_articles=result.get("referenced_articles", []),
            precedent_count=result.get("precedent_count"),
            related_keywords=result.get("related_keywords", [])
        )
        results.append(search_result)
    
    return results

# LLM 응답을 생성하는 함수
async def generate_legal_answer(query: str, driver, embedding_model) -> LegalResponse:
    # Agent 실행을 위한 컨텍스트 생성
    context = LegalContext(
        query=query,
        driver=driver,
        embedding_model=embedding_model
    )
    
    # Agent 실행
    result = await legal_agent.run(query, deps=context)
    return result.data

# 동기 버전 - Jupyter 노트북에서 사용 편의성 제공
def generate_legal_answer_sync(query: str, driver, embedding_model) -> LegalResponse:
    """법률 질의에 대한 응답을 생성합니다."""
    import asyncio
    
    # 비동기 함수를 동기적으로 실행
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        # 이미 이벤트 루프가 닫혔거나 없는 경우 새로 생성
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    
    result = loop.run_until_complete(generate_legal_answer(query, driver, embedding_model))
    return result

In [22]:
import pandas as pd
import json
from tqdm import tqdm

def create_batch_jsonl():
    # 1. CSV 파일 로드
    csv_path = "./dataset/Criminal-Law-test.csv"
    print(f"CSV 파일 로드 중: {csv_path}")
    try:
        df = pd.read_csv(csv_path)
        print(f"✅ {len(df)}개 문항 로드 완료")
    except Exception as e:
        print(f"❌ CSV 파일 로드 실패: {e}")
        return
    
    # 2. 메타데이터 저장 (평가에 필요한 정보)
    metadata = []
    
    # 3. JSONL 파일 생성
    output_file = "criminal_law_batch.jsonl"
    with open(output_file, "w", encoding="utf-8") as f:
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="JSONL 생성 중"):
            question = row["question"]
            choices = {
                "A": row["A"],
                "B": row["B"],
                "C": row["C"],
                "D": row["D"]
            }
            
            # 문제 포맷팅
            formatted_question = f"""
질문: {question}

A. {choices['A']}
B. {choices['B']}
C. {choices['C']}
D. {choices['D']}

위 질문의 정답을 A, B, C, D 중에서 하나만 선택해 주세요.
"""
            
            # Batch API 요청 형식
            request_data = {
                "custom_id": f"question-{idx}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": "gpt-4o-mini",
                    "messages": [
                        {"role": "system", "content": "당신은 한국 형법 전문가입니다. 주어진 문제의 정답을 A, B, C, D 중에서 선택하세요."},
                        {"role": "user", "content": formatted_question}
                    ],
                    "temperature": 0.0,
                    "max_tokens": 100
                }
            }
            
            # JSONL 형식으로 저장
            f.write(json.dumps(request_data, ensure_ascii=False) + "\n")
            
            # 메타데이터 저장
            metadata.append({
                "question_id": idx,
                "question": question,
                "answer": row["answer"]
            })
    
    # 4. 메타데이터 저장 (평가에 사용할 정답 정보)
    with open("criminal_law_metadata.json", "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSONL 파일 생성 완료: {output_file}")
    print(f"✅ 메타데이터 파일 생성 완료: criminal_law_metadata.json")
    
    return output_file

# 함수 실행
jsonl_file = create_batch_jsonl()

CSV 파일 로드 중: ./dataset/Criminal-Law-test.csv
✅ 200개 문항 로드 완료


JSONL 생성 중: 100%|█████████████████████████████████████| 200/200 [00:00<00:00, 23786.67it/s]

✅ JSONL 파일 생성 완료: criminal_law_batch.jsonl
✅ 메타데이터 파일 생성 완료: criminal_law_metadata.json


In [24]:
from openai import OpenAI
import time

# API 키 설정
client = OpenAI(api_key=OPENAI_API_KEY)

def submit_batch_job(jsonl_file):
    # 1. 파일 업로드
    print(f"파일 업로드 중: {jsonl_file}")
    with open(jsonl_file, "rb") as f:
        batch_file = client.files.create(
            file=f,
            purpose="batch"
        )
    print(f"✅ 파일 업로드 완료: {batch_file.id}")
    
    # 2. 배치 작업 생성
    batch = client.batches.create(
        input_file_id=batch_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h"
    )
    print(f"✅ 배치 작업 생성 완료: {batch.id}")
    
    return batch.id

def check_batch_status(batch_id):
    batch = client.batches.retrieve(batch_id)
    status = batch.status
    print(f"배치 상태: {status}")
    
    if status == "completed":
        print(f"✅ 배치 완료: 출력 파일 ID = {batch.output_file_id}")
        return batch.output_file_id
    return None

def download_results(output_file_id, result_file="batch_results.jsonl"):
    result = client.files.content(output_file_id)
    
    with open(result_file, "wb") as f:
        for chunk in result.iter_bytes():
            f.write(chunk)
    
    print(f"✅ 결과 파일 다운로드 완료: {result_file}")
    return result_file

# 배치 작업 제출
batch_id = submit_batch_job("criminal_law_batch.jsonl")

# 상태 확인 (완료될 때까지 대기)
output_file_id = None
while True:
    output_file_id = check_batch_status(batch_id)
    if output_file_id:
        break
    print("⏳ 60초 후 다시 확인...")
    time.sleep(60)

# 결과 다운로드
if output_file_id:
    results_file = download_results(output_file_id)

파일 업로드 중: criminal_law_batch.jsonl
✅ 파일 업로드 완료: file-6Qya1qh6YyAHgNMmUwd58i
✅ 배치 작업 생성 완료: batch_67f8f52b01a88190901773826a5f346b
배치 상태: validating
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: in_progress
⏳ 60초 후 다시 확인...
배치 상태: 

In [25]:
def evaluate_results(results_file, metadata_file):
    import json
    
    # 메타데이터 로드
    with open(metadata_file, "r", encoding="utf-8") as f:
        metadata = json.load(f)
    
    # 결과 파일 로드
    results = []
    with open(results_file, "r", encoding="utf-8") as f:
        for line in f:
            results.append(json.loads(line))
    
    # 평가
    correct = 0
    total = 0
    
    for result in results:
        # 결과 매칭 (custom_id로)
        custom_id = result.get("custom_id", "")
        idx = int(custom_id.split("-")[-1])
        
        if idx >= len(metadata):
            continue
            
        meta = metadata[idx]
        
        # 응답에서 답변 추출
        response_body = result.get("response", {}).get("body", {})
        content = response_body.get("choices", [{}])[0].get("message", {}).get("content", "")
        
        # 답변 추출 (A, B, C, D 중 하나)
        model_answer = extract_answer(content)
        correct_answer = meta["answer"]
        
        # 정확도 계산
        is_correct = (model_answer == correct_answer)
        if is_correct:
            correct += 1
        total += 1
    
    accuracy = correct / total if total > 0 else 0
    print(f"\n📊 평가 결과:")
    print(f"총 문항: {total}")
    print(f"정답: {correct}")
    print(f"정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")

# 답변 추출 함수
def extract_answer(content):
    content = content.upper()
    
    # 정답 패턴 확인
    if "정답은 A" in content or "정답: A" in content or "정답 A" in content:
        return "A"
    elif "정답은 B" in content or "정답: B" in content or "정답 B" in content:
        return "B"
    elif "정답은 C" in content or "정답: C" in content or "정답 C" in content:
        return "C"
    elif "정답은 D" in content or "정답: D" in content or "정답 D" in content:
        return "D"
    
    # 위 패턴이 없는 경우, 가장 먼저 나오는 A, B, C, D 찾기
    for char in content:
        if char in "ABCD":
            return char
    
    # 못 찾은 경우
    return "X"

# 평가 실행
evaluate_results("batch_results.jsonl", "criminal_law_metadata.json")


📊 평가 결과:
총 문항: 200
정답: 0
정확도: 0.0000 (0.00%)
